In [22]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os

In [23]:
from visualization import time_recovered, time_deaths, time_confirmed, data

In [24]:
data.rename({'Last Update': 'Date'}, axis=1, inplace=True)
data['Date']=pd.to_datetime(data['Date'])
data.head()

,Province/State,Country/Region,Date,Confirmed,Deaths,Recovered,Latitude,Longitude,Location
0,Hubei,Mainland China,2020-03-07 11:13:04,67666,2959,43500,30.9756,112.2707,"(30.9756, 112.2707)"
1,NaN,South Korea,2020-03-07 10:13:22,7041,44,135,36.0000,128.0000,"(36.0, 128.0)"
2,NaN,Italy,2020-03-07 17:33:03,5883,233,589,43.0000,12.0000,"(43.0, 12.0)"
3,NaN,Iran,2020-03-07 17:43:05,5823,145,1669,32.0000,53.0000,"(32.0, 53.0)"
4,Guangdong,Mainland China,2020-03-07 10:43:02,1352,7,1237,23.3417,113.4244,"(23.3417, 113.4244)"


In [25]:
lat=data['Latitude'].tolist()
long=data['Longitude'].tolist()

In [26]:
time_recovered.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20
0,NaN,Thailand,15.0000,101.0000,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
1,NaN,Japan,36.0000,138.0000,0,0,0,0,0,0,...,10,10,15,16,19,22,22,27,29,29
2,NaN,Singapore,1.2833,103.8333,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,Nepal,28.1667,84.2500,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,Malaysia,2.5000,112.5000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2


In [27]:
location=[]
for idx in data.index:
    loc=data.iloc[idx]['Latitude'],data.iloc[idx]['Longitude']
    location.append(loc)

In [28]:
location_series=pd.Series(location)
data['Location'] = location_series

In [29]:
data.head()

,Province/State,Country/Region,Date,Confirmed,Deaths,Recovered,Latitude,Longitude,Location
0,Hubei,Mainland China,2020-03-07 11:13:04,67666,2959,43500,30.9756,112.2707,"(30.9756, 112.2707)"
1,NaN,South Korea,2020-03-07 10:13:22,7041,44,135,36.0000,128.0000,"(36.0, 128.0)"
2,NaN,Italy,2020-03-07 17:33:03,5883,233,589,43.0000,12.0000,"(43.0, 12.0)"
3,NaN,Iran,2020-03-07 17:43:05,5823,145,1669,32.0000,53.0000,"(32.0, 53.0)"
4,Guangdong,Mainland China,2020-03-07 10:43:02,1352,7,1237,23.3417,113.4244,"(23.3417, 113.4244)"


In [30]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from IPython.display import Image

In [31]:
loc_dir=os.path.join(os.path.dirname(os.getcwd()),"COVID-19/all_countries.csv")
loc_df=pd.read_csv(loc_dir)
loc_df=loc_df.fillna("Other")
loc_df.head()

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,Other,142,34,Other
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,Other,150,154,Other
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,Other,150,39,Other
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,Other,2,15,Other
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,Other,9,61,Other


### Join location to all DFs

In [32]:
merge_loc=[]
for d in [time_recovered, time_deaths, time_confirmed, data]:
    d=d.merge(loc_df, left_on='Country/Region', right_on='name')
    merge_loc.append(d)
time_recovered, time_deaths, time_confirmed, data=merge_loc[0], merge_loc[1], merge_loc[2], merge_loc[3]

In [33]:
data.head()

,Province/State,Country/Region,Date,Confirmed,Deaths,Recovered,Latitude,Longitude,Location,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,NaN,Italy,2020-03-07 17:33:03,5883,233,589,43.0,12.0,"(43.0, 12.0)",Italy,IT,ITA,380,ISO 3166-2:IT,Europe,Southern Europe,Other,150,39,Other
1,NaN,France,2020-03-07 20:33:02,949,11,12,47.0,2.0,"(47.0, 2.0)",France,FR,FRA,250,ISO 3166-2:FR,Europe,Western Europe,Other,150,155,Other
2,NaN,Germany,2020-03-07 17:43:05,799,0,18,51.0,9.0,"(51.0, 9.0)",Germany,DE,DEU,276,ISO 3166-2:DE,Europe,Western Europe,Other,150,155,Other
3,NaN,Spain,2020-03-07 17:43:05,500,10,30,40.0,-4.0,"(40.0, -4.0)",Spain,ES,ESP,724,ISO 3166-2:ES,Europe,Southern Europe,Other,150,39,Other
4,NaN,Japan,2020-03-07 17:43:05,461,6,76,36.0,138.0,"(36.0, 138.0)",Japan,JP,JPN,392,ISO 3166-2:JP,Asia,Eastern Asia,Other,142,30,Other


### Dig Now/Relenthless Later drop for now NaN values

In [34]:
time_recovered=time_recovered.fillna("Other") 
time_deaths=time_deaths.fillna("Other") 
time_confirmed=time_confirmed.fillna("Other")
data=data.fillna("Other")

In [35]:
column=time_deaths.columns

In [36]:
#time_recovered=time_recovered.groupby('Country/Region')[column[15:]].sum()
#time_deaths=time_deaths.groupby('Country/Region')[column[15:]].sum()
#time_confirmed=time_confirmed.groupby('Country/Region')[column[15:]].sum()

In [37]:
data.head()

,Province/State,Country/Region,Date,Confirmed,Deaths,Recovered,Latitude,Longitude,Location,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Other,Italy,2020-03-07 17:33:03,5883,233,589,43.0,12.0,"(43.0, 12.0)",Italy,IT,ITA,380,ISO 3166-2:IT,Europe,Southern Europe,Other,150.0,39.0,Other
1,Other,France,2020-03-07 20:33:02,949,11,12,47.0,2.0,"(47.0, 2.0)",France,FR,FRA,250,ISO 3166-2:FR,Europe,Western Europe,Other,150.0,155.0,Other
2,Other,Germany,2020-03-07 17:43:05,799,0,18,51.0,9.0,"(51.0, 9.0)",Germany,DE,DEU,276,ISO 3166-2:DE,Europe,Western Europe,Other,150.0,155.0,Other
3,Other,Spain,2020-03-07 17:43:05,500,10,30,40.0,-4.0,"(40.0, -4.0)",Spain,ES,ESP,724,ISO 3166-2:ES,Europe,Southern Europe,Other,150.0,39.0,Other
4,Other,Japan,2020-03-07 17:43:05,461,6,76,36.0,138.0,"(36.0, 138.0)",Japan,JP,JPN,392,ISO 3166-2:JP,Asia,Eastern Asia,Other,142.0,30.0,Other


In [38]:
loc_list=[]
for idx in data.index:
    loc_obj={
      "text": "Geo-point as an object",
      "location": { 
        "lat": data.iloc[idx]['Latitude'],
        "lon": data.iloc[idx]['Longitude']
      }
    }
    loc_list.append(loc_obj)
df_loc=pd.DataFrame(loc_list)
data=pd.merge(data, df_loc, left_index=True, right_index=True)

In [39]:
'''
def import_to_elastic(df,t):
    
    idx='covid'+'_'+t
    doc_t=t+"_cases"
    
    es = Elasticsearch(["127.0.0.1:9200"])
    es.indices.delete(index=idx,ignore=404)  #if index exist delete it, or ignore error messages, 404=index not found
    docs = df.to_dict(orient='records')       #from dataset create a serialize object for import 
    bulk(es, docs, index=idx,doc_type=doc_t, raise_on_error=True) #bulk import
    es.indices.refresh() #get import status
    
    return(es)

df_list=[time_recovered,time_deaths,time_confirmed,data]
title=['recovered','deaths','confirmed','data']

for d,t in zip(df_list, title):
    import_to_elastic(d,t)
'''

'\ndef import_to_elastic(df,t):\n    \n    idx=\'covid\'+\'_\'+t\n    doc_t=t+"_cases"\n    \n    es = Elasticsearch(["127.0.0.1:9200"])\n    es.indices.delete(index=idx,ignore=404)  #if index exist delete it, or ignore error messages, 404=index not found\n    docs = df.to_dict(orient=\'records\')       #from dataset create a serialize object for import \n    bulk(es, docs, index=idx,doc_type=doc_t, raise_on_error=True) #bulk import\n    es.indices.refresh() #get import status\n    \n    return(es)\n\ndf_list=[time_recovered,time_deaths,time_confirmed,data]\ntitle=[\'recovered\',\'deaths\',\'confirmed\',\'data\']\n\nfor d,t in zip(df_list, title):\n    import_to_elastic(d,t)\n'

In [43]:
data.head()

,Province/State,Country/Region,Date,Confirmed,Deaths,Recovered,Latitude,Longitude,Location,name,...,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code,text,location
0,Other,Italy,2020-03-07 17:33:03,5883,233,589,43.0,12.0,"(43.0, 12.0)",Italy,...,380,ISO 3166-2:IT,Europe,Southern Europe,Other,150.0,39.0,Other,Geo-point as an object,"{'lat': 43.0, 'lon': 12.0}"
1,Other,France,2020-03-07 20:33:02,949,11,12,47.0,2.0,"(47.0, 2.0)",France,...,250,ISO 3166-2:FR,Europe,Western Europe,Other,150.0,155.0,Other,Geo-point as an object,"{'lat': 47.0, 'lon': 2.0}"
2,Other,Germany,2020-03-07 17:43:05,799,0,18,51.0,9.0,"(51.0, 9.0)",Germany,...,276,ISO 3166-2:DE,Europe,Western Europe,Other,150.0,155.0,Other,Geo-point as an object,"{'lat': 51.0, 'lon': 9.0}"
3,Other,Spain,2020-03-07 17:43:05,500,10,30,40.0,-4.0,"(40.0, -4.0)",Spain,...,724,ISO 3166-2:ES,Europe,Southern Europe,Other,150.0,39.0,Other,Geo-point as an object,"{'lat': 40.0, 'lon': -4.0}"
4,Other,Japan,2020-03-07 17:43:05,461,6,76,36.0,138.0,"(36.0, 138.0)",Japan,...,392,ISO 3166-2:JP,Asia,Eastern Asia,Other,142.0,30.0,Other,Geo-point as an object,"{'lat': 36.0, 'lon': 138.0}"


In [40]:
#gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data.Longitude, data.Latitude))
#gdf

In [42]:
es = Elasticsearch(["127.0.0.1:9200"])
es.indices.delete(index='geotest',ignore=404)  #if index exist delete it, or ignore error messages, 404=index not found
docs = data.to_dict(orient='records')       #from dataset create a serialize object for import 
bulk(es, docs, index='geotest',doc_type='geo_data', raise_on_error=True) #bulk import
es.indices.refresh() #get import status

print(es)

<Elasticsearch([{'host': '127.0.0.1', 'port': 9200}])>
